In [1]:
# train_resnet_model.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 40
BATCH_SIZE = 32
IMG_SIZE = 224

TRAIN_CSV = "./data/train.csv"
VAL_CSV = "./data/val.csv"
TEST_CSV = "./data/test.csv"
TRAIN_DIR = "./data/train"
VAL_DIR = "./data/val"
TEST_DIR = "./data/test"
MODEL_SAVE_PATH = "./resnet_based_models"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

class PhotoDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.data.iloc[idx, 1]
        return image, label

# Data augmentation + normalization
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = PhotoDataset(TRAIN_CSV, TRAIN_DIR, transform)
val_dataset = PhotoDataset(VAL_CSV, VAL_DIR, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Compute class weights
y_train = pd.read_csv(TRAIN_CSV)['label'].values
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

# Load pretrained ResNet18 and modify classifier
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)  # Binary classification
)
model = resnet.to(DEVICE)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs = inputs.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

    # Save model
    if (epoch + 1) % 5 == 0:
        save_path = os.path.join(MODEL_SAVE_PATH, f"resnet_model_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved at {save_path}")


/Users/mradula.singh/Library/Python/3.12/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mradula.singh/Library/Python/3.12/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/40: 100%|██████████| 125/125 [05:19<00:00,  2.55s/it]


Epoch 1: Loss = 0.8218


Epoch 2/40: 100%|██████████| 125/125 [05:08<00:00,  2.47s/it]


Epoch 2: Loss = 0.6288


Epoch 3/40: 100%|██████████| 125/125 [05:15<00:00,  2.52s/it]


...

Epoch 38: Loss = 0.1053


Epoch 39/40: 100%|██████████| 125/125 [03:36<00:00,  1.73s/it]


Epoch 39: Loss = 0.0473


Epoch 40/40: 100%|██████████| 125/125 [03:31<00:00,  1.69s/it]

Epoch 40: Loss = 0.1310
✅ Model saved at ./resnet_based_models/resnet_model_epoch_40.pth


## Oversampling based approach

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 40
BATCH_SIZE = 32
IMG_SIZE = 224

# Paths
TRAIN_CSV = "./data/train.csv"
VAL_CSV = "./data/val.csv"
TRAIN_DIR = "./data/train"
VAL_DIR = "./data/val"
MODEL_SAVE_PATH = "./resnet_based_models_oversampling"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

# Load train & val CSVs
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)

# Get all label==1 from val and add to train
val_label_1 = val_df[val_df["label"] == 1].copy()
val_label_1["source"] = "val"
train_df["source"] = "train"
combined_df = pd.concat([train_df, val_label_1], ignore_index=True)

# Now balance the dataset (upsample minority)
label_counts = combined_df["label"].value_counts()
print(f"Before balancing:\n{label_counts}")

min_class = combined_df[combined_df["label"] == 1]
maj_class = combined_df[combined_df["label"] == 0]

if len(min_class) < len(maj_class):
    # Upsample minority
    min_class_upsampled = min_class.sample(len(maj_class), replace=True, random_state=42)
    balanced_df = pd.concat([maj_class, min_class_upsampled], ignore_index=True)
    print(f"✅ Upsampled label=1 to match label=0: {len(min_class_upsampled)} samples.")
else:
    # Upsample majority (rare case)
    maj_class_upsampled = maj_class.sample(len(min_class), replace=True, random_state=42)
    balanced_df = pd.concat([min_class, maj_class_upsampled], ignore_index=True)

# Shuffle dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save balanced dataset for tracking
balanced_df.to_csv("train_balanced.csv", index=False)

# Dataset class
class PhotoDataset(Dataset):
    def __init__(self, dataframe, img_dirs, transform=None):
        self.data = dataframe
        self.img_dirs = img_dirs
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_dir = self.img_dirs[row['source']]
        img_path = os.path.join(img_dir, row['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = row['label']
        return image, label

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Datasets
train_dataset = PhotoDataset(balanced_df, {"train": TRAIN_DIR, "val": VAL_DIR}, train_transform)
val_dataset = PhotoDataset(val_df, {"val": VAL_DIR}, val_transform)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Class weights (should be 1:1 now, but still good practice)
y_train = balanced_df['label'].values
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

# Model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
)
model = resnet.to(DEVICE)

# Loss & Optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs = inputs.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1}: Loss = {avg_loss:.4f}")

    # Save model every 5 epochs
    if (epoch + 1) % 2 == 0:
        save_path = os.path.join(MODEL_SAVE_PATH, f"resnet_model_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved at {save_path}")


/Users/mradula.singh/Library/Python/3.12/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mradula.singh/Library/Python/3.12/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Before balancing:
label
0    3800
1     216
Name: count, dtype: int64
✅ Upsampled label=1 to match label=0: 3800 samples.


Epoch 1/40: 100%|██████████| 238/238 [06:27<00:00,  1.63s/it]


📘 Epoch 1: Loss = 0.3794


Epoch 2/40: 100%|██████████| 238/238 [06:28<00:00,  1.63s/it]


...

📘 Epoch 12: Loss = 0.0188
✅ Model saved at ./resnet_based_models_oversampling/resnet_model_epoch_12.pth


Epoch 13/40: 100%|██████████| 238/238 [06:52<00:00,  1.73s/it]


📘 Epoch 13: Loss = 0.0242


Epoch 14/40:  72%|███████▏  | 171/238 [04:49<01:53,  1.69s/it]


KeyboardInterrupt: 